[View in Colaboratory](https://colab.research.google.com/github/cahya-wirawan/ML-Collection/blob/master/Text_Classification_Amazon_Review_TPU.ipynb)

In [2]:
#!pip install textblob
#!pip install -U ntlk

    100% |████████████████████████████████| 634kB 8.7MB/s 


In [0]:
from sklearn import model_selection, preprocessing
from sklearn import linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import tensorflow as tf
#import pandas as pd, xgboost, numpy, textblob, string
import pandas as pd, xgboost, numpy, string
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import layers, models, optimizers
from keras.preprocessing.sequence import pad_sequences
from pathlib import Path

import os
import numpy as np
#import ntlk

In [29]:
# Parameters
LMDATA = Path('/content/drive/My Drive/lmdata')
params = {'batch_size': 1024,
          'n_classes': 2,
          'max_len': 100,
          'n_words': 50000,
          'shuffle': True}
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER))

KeyError: ignored

In [0]:
#from google.colab import files

#uploaded = files.upload()

#for fn in uploaded.keys():
#  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [4]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [30]:
!pwd
print(LMDATA)
!ls -lh "$LMDATA"
#!ls -l "/content/"

/content
/content/drive/My Drive/lmdata
total 2.3G
-rw------- 1 root root 4.3M Apr 21 23:28 amazon-review.csv
-rw------- 1 root root  58M Oct  8 16:03 bard-1.h5
-rw------- 1 root root  39M Oct  9 10:32 bard.h5
-rw------- 1 root root  22M Oct  8 13:31 wiki-news-300d-10K.vec
-rw------- 1 root root 2.2G Mar 14  2018 wiki-news-300d-1M.vec


In [0]:
#df = pd.read_csv('amazon-review.csv')
#drive/My Drive/lmdata/amazon-review.csv
#LMDATA = "/content/drive/My Drive/lmdata"

In [0]:


# load the dataset
data = open(LMDATA/'amazon-review.csv').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split(" ", 1)
    labels.append(content[0])
    texts.append(content[1])

# create a dataframe using texts and lables
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels


In [32]:
print(trainDF.head())
print(texts[:2])

                                                text       label
0  Stuning even for the non-gamer: This sound tra...  __label__2
1  The best soundtrack ever to anything.: I'm rea...  __label__2
2  Amazing!: This soundtrack is my favorite music...  __label__2
3  Excellent Soundtrack: I truly like this soundt...  __label__2
4  Remember, Pull Your Jaw Off The Floor After He...  __label__2
['Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^', "The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This i

In [6]:
#!pip show tensorflow
#!nvidia-smi
!set |grep -i tpu|grep -v grep

COLAB_TPU_ADDR=10.20.112.162:8470
DATALAB_SETTINGS_OVERRIDES='{"datalabBasePath":"/tun/m/tpu-2rjvnxs4v9jri/","kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0.3","jupyterArgs":["notebook","-y","--no-browser","--log-level=DEBUG","--debug","--NotebookApp.allow_origin=\"*\"","--NotebookApp.log_format=\"%(message)s\"","--NotebookApp.disable_check_xsrf=True","--NotebookApp.token=","--Session.key=\"\"","--Session.keyfile=\"\"","--ContentsManager.untitled_directory=\"Untitled Folder\"","--ContentsManager.untitled_file=\"Untitled File\"","--ContentsManager.untitled_notebook=\"Untitled Notebook\"","--KernelManager.autorestart=True","--ip=\"172.28.0.2\""]}'


In [0]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [8]:
print(train_x[:5])
print(train_y[:5])
print(valid_x[:5])
print(valid_y[:5])


4801    Not so great: This is NOT Koss at its best. I ...
3163    SLOW very very slow: Know this the web site fo...
4222    Seems to be good!: This seems to be good so fa...
4945    Book by Dale Carnegie: I read book Quick and E...
2677    Disappointing: This poor character never catch...
Name: text, dtype: object
[0 0 1 1 0]
6204    You will FALL IN LOVE with Auel's Ayla!!: You ...
6165    Looks like it was Printed on a cheap broken do...
5509    I used to eat it till I found out how it was m...
2465    I made better recordings in my garage: I was e...
9348    One of the all time best reads ever!: I read t...
Name: text, dtype: object
[1 0 0 0 1]


In [0]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [0]:
#print(xtrain_count, xvalid_count)


In [0]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                             max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                                   ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}',
                                         ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [35]:
# load the pre-trained word-embedding vectors
max_words = params['n_words']
embeddings_index = {}
for i, line in enumerate(open('drive/My Drive/lmdata/wiki-news-300d-10K.vec')):
  if i%1000 == 0:
    print(i)
  values = line.split()
  embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    if i>=max_words:
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [0]:
trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
trainDF['upper_case_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [36]:
print(trainDF.head())

                                                text       label
0  Stuning even for the non-gamer: This sound tra...  __label__2
1  The best soundtrack ever to anything.: I'm rea...  __label__2
2  Amazing!: This soundtrack is my favorite music...  __label__2
3  Excellent Soundtrack: I truly like this soundt...  __label__2
4  Remember, Pull Your Jaw Off The Floor After He...  __label__2


In [0]:
#!pip install -U nltk
#import ntlk
#ntlk.download('punkt')

In [0]:
"""
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

trainDF['noun_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'noun'))
trainDF['verb_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'verb'))
trainDF['adj_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adj'))
trainDF['adv_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adv'))
trainDF['pron_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'pron'))
"""

In [0]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [0]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, 
                is_neural_net=False, epochs=1):
    # fit the training dataset on the classifier
    if is_neural_net:
      classifier.fit(feature_vector_train, label, epochs=epochs)
    else:
      classifier.fit(feature_vector_train, label)   
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    
    if is_neural_net:
      predictions = [int(round(p[0])) for p in predictions]
      #predictions = predictions.argmax(axis=-1)

    print("predictions", predictions[:20])
    print("valid_y", valid_y[:20])
    
    return metrics.accuracy_score(predictions, valid_y)

In [14]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)

predictions [1 0 0 0 1 0 1 1 1 0 0 0 1 0 0 0 0 1 1 0]
valid_y [1 0 0 0 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1]
NB, Count Vectors:  0.838
predictions [1 0 0 0 1 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0]
valid_y [1 0 0 0 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1]
NB, WordLevel TF-IDF:  0.8508
predictions [1 0 0 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 1 0]
valid_y [1 0 0 0 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1]
NB, N-Gram Vectors:  0.8352
predictions [1 0 0 0 1 0 1 1 1 0 0 0 1 0 0 0 0 1 1 0]
valid_y [1 0 0 0 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1]
NB, CharLevel Vectors:  0.8052


In [15]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print( "LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)

predictions [1 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 1 1]
valid_y [1 0 0 0 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1]
LR, Count Vectors:  0.8648
predictions [1 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 0 1 1 1]
valid_y [1 0 0 0 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1]
LR, WordLevel TF-IDF:  0.8728
predictions [1 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 1 1 0]
valid_y [1 0 0 0 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1]
LR, N-Gram Vectors:  0.8308
predictions [1 0 1 0 1 1 1 0 1 0 0 0 1 0 0 1 0 1 1 1]
valid_y [1 0 0 0 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1]
LR, CharLevel Vectors:  0.8356


In [0]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)


predictions [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
valid_y [0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0]
SVM, N-Gram Vectors:  0.5152


In [0]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print("RF, WordLevel TF-IDF: ", accuracy)

predictions [0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0]
valid_y [0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0]
RF, Count Vectors:  0.744
predictions [0 0 1 0 1 0 0 0 1 0 0 0 0 1 1 0 1 1 1 0]
valid_y [0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0]
RF, WordLevel TF-IDF:  0.766


In [0]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print("Xgb, Count Vectors: ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print("Xgb, WordLevel TF-IDF: ", accuracy)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print("Xgb, CharLevel Vectors: ", accuracy)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


predictions [0 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1 0]
valid_y [0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0]
Xgb, Count Vectors:  0.8052


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


predictions [0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 0 1 0 1 0]
valid_y [0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0]
Xgb, WordLevel TF-IDF:  0.8032
predictions [0 0 1 0 1 0 1 1 1 1 0 1 0 0 1 0 1 0 1 0]
valid_y [0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0]
Xgb, CharLevel Vectors:  0.806


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [0]:
from tensorflow.keras.utils import Sequence

def tokenize(texts, n_words=1000):
    tokenizer = Tokenizer(num_words=n_words)
    tokenizer.fit_on_texts(texts)
    return tokenizer
  
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, texts, labels, tokenizer, batch_size=32, max_len=100,
                 n_classes=2, n_words=1000, shuffle=True):
        'Initialization'
        self.max_len = max_len
        self.batch_size = batch_size
        self.texts = texts
        self.labels = labels
        self.n_classes = n_classes
        self.shuffle = shuffle
        print(">> Init text:", self.texts[0])

        self.tokenizer = tokenizer
        self.steps_per_epoch = int(np.floor(self.texts.size / self.batch_size))
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.steps_per_epoch

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        texts = np.array([self.texts[k] for k in indexes])
        sequences = self.tokenizer.texts_to_sequences(texts)
        X = pad_sequences(sequences, maxlen=self.max_len)
        y = np.array([self.labels[k] for k in indexes])

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        #print(">> on_epoch_end:")
        self.indexes = np.arange(self.texts.size)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [0]:
%%time

def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu", name="D1")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, 
                       is_neural_net=True, epochs=1)
print("NN, Ngram Level TF IDF Vectors",  accuracy)

In [38]:
xtrain_tfidf_ngram.shape

(7500, 5000)

In [0]:
def to_tpu(func):
  def wrapper(*args, **kwargs):
    print("TPU Wrapper start")
    print(func)
    TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
          tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER))
    tpu_model = tf.contrib.tpu.keras_to_tpu_model(func, strategy)
    print("TPU Wrapper end")
    return tpu_model(*args, **kwargs)
  return wrapper

In [0]:
def create_cnn(input_length=100):
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, 
                                       weights=[embedding_matrix], 
                                       trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.3)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [40]:
# Create data generator
print("training_generator:{}, {}".format(type(train_x), train_y))
training_generator = DataGenerator(train_x.values, train_y, token, **params)
print("valid_generator:{}".format(type(valid_x)))
valid_generator = DataGenerator(valid_x.values, valid_y, token,  **params)
print("read generator")


training_generator:<class 'pandas.core.series.Series'>, [1 1 0 ... 1 1 1]
>> Init text: Fantastic. Up close and personal. Scripturally accurate.: Doesn't get much better than this. What a blessing I have recieved from Pink's expositional approach to this great King's life.
valid_generator:<class 'pandas.core.series.Series'>
>> Init text: So Far So Good!: I've only worn this for a day so far, but I love it! I still have a little bit of a tummy from the birth of my son 8 months ago. This really holds you in, its comfortable and I don't think it shows through your clothes! I highly recommend!!
read generator


In [42]:
%%time

tf.keras.backend.clear_session()
classifier = create_cnn(input_length=params['max_len'])
#classifier = tf.contrib.tpu.keras_to_tpu_model(classifier, strategy=strategy)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

classifier.save_weights(str(LMDATA/'bard.h5'), overwrite=True)


Epoch 1/20
7/7 [==============================] - 3s 360ms/step - loss: 0.6963 - acc: 0.5166 - val_loss: 0.6824 - val_acc: 0.6504
Epoch 2/20
7/7 [==============================] - 1s 180ms/step - loss: 0.6799 - acc: 0.5818 - val_loss: 0.6656 - val_acc: 0.7285
Epoch 3/20
7/7 [==============================] - 1s 137ms/step - loss: 0.6603 - acc: 0.6394 - val_loss: 0.6413 - val_acc: 0.7495
Epoch 4/20
7/7 [==============================] - 1s 158ms/step - loss: 0.6351 - acc: 0.6921 - val_loss: 0.6061 - val_acc: 0.7710
Epoch 5/20
7/7 [==============================] - 1s 158ms/step - loss: 0.5980 - acc: 0.7303 - val_loss: 0.5571 - val_acc: 0.7847
Epoch 6/20
7/7 [==============================] - 1s 133ms/step - loss: 0.5501 - acc: 0.7592 - val_loss: 0.5031 - val_acc: 0.7915
Epoch 7/20
7/7 [==============================] - 1s 160ms/step - loss: 0.4998 - acc: 0.7881 - val_loss: 0.4583 - val_acc: 0.8032
Epoch 8/20
7/7 [==============================] - 1s 143ms/step - loss: 0.4557 - acc: 0.80

In [0]:

classifier.save_weights(str(LMDATA/'bard.h5'), overwrite=True)

In [46]:
# predict the labels on validation dataset


sequences = token.texts_to_sequences(valid_x.values)
valid_x_seq = pad_sequences(sequences, maxlen=params['max_len'])
#print(valid_x_seq[:5])
classifier = create_cnn(input_length=params['max_len'])
classifier.load_weights(str(LMDATA/'bard.h5'))
"""
strategy = tf.contrib.tpu.TPUDistributionStrategy(
    tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER))
prediction_model = tf.contrib.tpu.keras_to_tpu_model(
    prediction_model, strategy=strategy)
"""
score = classifier.evaluate(valid_x_seq, valid_y, verbose=1)
print('Loss for final step: {}, accuracy: {}'.format(score[0], score[1]))




2500/2500 [==============================] - 1s 248us/step
Loss for final step: 0.3150304111480713, accuracy: 0.86


In [47]:
score = classifier.predict(valid_x_seq, verbose=1)
print('Loss for final step: {}'.format(np.round(score).astype(int).reshape(-1)[:5]))
print(valid_x[:5])

2500/2500 [==============================] - 0s 93us/step
Loss for final step: [1 1 1 1 1]
642     So Far So Good!: I've only worn this for a day...
8690    GOOD LOOKIN'!!: My girlfriend told me I had to...
6335    Great Game.... :): This is a game everyone sho...
6605    Great Value Priced CD: Saw a TV Show not long ...
515     Back to basics...: Sometimes back to basics is...
Name: text, dtype: object


In [21]:
!free

              total        used        free      shared  buff/cache   available
Mem:       13335236     1136812     8625904         804     3572520    12333120
Swap:             0           0           0


In [0]:

#print('Accuracy ', score[1])
"""
# First, run the seed forward to prime the state of the model.
#prediction_model.reset_states()
strategy = tf.contrib.tpu.TPUDistributionStrategy(
    tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER))
prediction_model = tf.contrib.tpu.keras_to_tpu_model(
    prediction_model, strategy=strategy)

predictions = prediction_model.predict(valid_x_seq)

print("predictions", predictions[:20])

predictions = tpu_model.predict(valid_x_seq)

predictions = [int(round(p[0])) for p in predictions]
#predictions = predictions.argmax(axis=-1)

print("predictions", predictions[:20])
print("valid_y", valid_y[:20])

return metrics.accuracy_score(predictions, valid_y)
"""

In [22]:
print(score)

[[0.07394564]
 [0.96206063]
 [0.9657972 ]
 ...
 [0.00592667]
 [0.35479817]
 [0.11346152]]


# New Section

In [0]:
def create_rnn_lstm(input_length=100):
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [32]:
%%time

tf.keras.backend.clear_session()
classifier = create_rnn_lstm(input_length=params['max_len'])
classifier = tf.contrib.tpu.keras_to_tpu_model(classifier, strategy=strategy)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

Epoch 1/20
INFO:tensorflow:New input shapes; (re-)compiling: mode=train, [TensorSpec(shape=(128, 100), dtype=tf.float32, name='input_10'), TensorSpec(shape=(128, 1), dtype=tf.float32, name='dense_1_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
INFO:tensorflow:Cloning Adam {'lr': 0.0010000000474974513, 'beta_1': 0.8999999761581421, 'beta_2': 0.9990000128746033, 'decay': 0.0, 'epsilon': 1e-07, 'amsgrad': False}
INFO:tensorflow:Get updates: Tensor("loss_12/mul:0", shape=(), dtype=float32)
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 7.8834638595581055 secs
INFO:tensorflow:Setting weights on TPU model.
6/7 [========================>.....] - ETA: 2s - loss: 0.6912 - acc: 0.5078INFO:tensorflow:New input shapes; (re-)compiling: mode=eval, [TensorSpec(shape=(128, 100), dtype=tf.float32, name='input_10'), TensorSpec(shape=(128, 1), dtype=tf.float32, name='dense_1_target_10')]
INFO:tensorflow:

In [27]:
%%time

def create_rnn_gru():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

classifier = create_rnn_gru()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, 
                       is_neural_net=True, epochs=10)
print("RNN-GRU, Word Embeddings",  accuracy)

Epoch 1/10
4128/7500 [===============>..............] - ETA: 19s - loss: 0.6575

KeyboardInterrupt: ignored

In [0]:
%%time

def create_bidirectional_rnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

classifier = create_bidirectional_rnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, 
                       is_neural_net=True, epochs=1)
print("RNN-Bidirectional, Word Embeddings",  accuracy)

Epoch 1/1
7500/7500 [==============================] - 38s 5ms/step - loss: 0.6029
predictions [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1]
valid_y [0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0]
RNN-Bidirectional, Word Embeddings 0.7936
CPU times: user 1min 21s, sys: 1.16 s, total: 1min 22s
Wall time: 45.7 s


In [0]:
%%time

def create_rcnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

classifier = create_rcnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, 
                       is_neural_net=True, epochs=10)
print("CNN, Word Embeddings",  accuracy)

Epoch 1/10
7500/7500 [==============================] - 12s 2ms/step - loss: 0.5678
Epoch 2/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.3829
Epoch 3/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.3374
Epoch 4/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.2814
Epoch 5/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.2365
Epoch 6/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.2043
Epoch 7/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.1675
Epoch 8/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.1320
Epoch 9/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.1081
Epoch 10/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.1005
predictions [0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1]
valid_y [0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0]
CNN, Word Embeddings 0.8624
CPU time

# New Section